### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6909,2023-11-29,Brasil Nbb,19:00,Botafogo,Bauru,3.61,1.26,156.5,1.87,1.79,7.5,2.00,1.72,rulgzuHk,0.277008,0.793651,0.534759,0.558659,0.070659,0.4,0.6,NaN,NaN,398.374,332.081192,0.833592,177.088,87.991780,0.496882,317.470,181.720,122.10,281.78,0.0,0.0,0.0,0.0,0.682423,0.030912,0.106446,-3.35,-0.67,-3.895522,0.000000,0.0,0.000000,-5.00,-1.000,-0.260000,0.000000,0.0,0.000000
6910,2023-11-29,Brasil Nbb,19:00,Minas,Unifacisa,1.25,3.69,155.5,1.87,1.79,-9.5,1.98,1.74,CdcNu1gS,0.800000,0.271003,0.534759,0.558659,0.071003,0.0,0.2,NaN,NaN,122.338,19.025097,0.155513,128.514,44.780985,0.348452,0.000,127.760,120.70,115.05,0.0,0.0,0.0,0.0,0.698518,0.030912,0.091240,0.00,0.00,inf,0.000000,0.0,0.000000,-1.87,-0.374,-7.192513,0.000000,0.0,0.000000
6911,2023-11-29,Brasil Nbb,19:30,Pato,Fortaleza B.C.,2.31,1.57,150.5,1.87,1.79,2.5,1.97,1.75,4lkkyL1q,0.432900,0.636943,0.534759,0.558659,0.069843,0.0,0.0,NaN,NaN,291.834,144.100690,0.493776,206.908,108.512863,0.524450,0.000,0.000,543.75,378.40,1.0,0.0,0.0,0.0,0.269721,0.030912,0.083636,0.00,0.00,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6912,2023-11-29,Eua Nba,00:00,Sacramento Kings,Golden State Warriors,1.82,2.15,235.5,1.88,2.02,-2.5,2.04,1.88,SvxQ6mZE,0.549451,0.465116,0.531915,0.495050,0.014567,0.4,0.6,NaN,NaN,221.632,57.587307,0.259833,212.146,54.590806,0.257327,205.492,220.574,265.32,258.75,0.0,0.0,0.0,0.0,0.117554,0.050767,0.057723,0.45,0.09,9.111111,0.544379,0.5,-0.044379,-2.17,-0.434,-2.649770,0.492038,0.4,-0.092038
6913,2023-11-29,Eua Nba,21:00,Detroit Pistons,Los Angeles Lakers,3.80,1.33,227.5,1.80,2.01,5.5,2.10,1.68,bL9iGngD,0.263158,0.751880,0.555556,0.497512,0.015038,0.8,0.6,NaN,NaN,338.398,99.047644,0.292696,193.090,54.783637,0.283721,310.138,211.292,177.62,269.78,0.0,0.0,0.0,0.0,0.680918,0.077949,0.157135,-5.00,-1.00,-2.800000,0.309146,0.1,-0.209146,-1.52,-0.304,-1.085526,0.455928,0.2,-0.255928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7093,2023-11-29,Europa Liga Waba Feminina,14:00,Plamen Pozega F,Partizan F,4.18,1.20,148.5,1.81,1.85,10.5,1.87,1.79,nsEcDLnb,0.239234,0.833333,0.552486,0.540541,0.072568,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,614.80,0.00,1.0,0.0,0.0,0.0,0.783338,0.015456,0.030912,0.00,0.00,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7094,2023-11-29,Noruega Blno,15:00,Gimle,Fyllingen,6.41,1.09,169.5,1.85,1.85,12.5,1.95,1.74,8n7r1j2e,0.156006,0.917431,0.540541,0.540541,0.073437,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,242.97,0.00,0.0,0.0,0.0,0.0,1.003149,0.000000,0.080484,0.00,0.00,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7095,2023-11-29,Rússia Copa Da Rússia,09:30,Novosibirsk,MBA Moscow,8.40,24.00,158.5,1.80,1.92,13.5,2.07,1.65,0SOIhLRt,0.119048,0.041667,0.555556,0.520833,-0.839286,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.680918,0.045620,0.159669,0.00,0.00,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7096,2023-11-29,Rússia Copa Da Rússia,10:00,Temp Sumz Revda,Samara,3.85,17.00,162.5,1.86,1.86,6.5,2.04,1.66,rZMic1JP,0.259740,0.058824,0.537634,0.537634,-0.681436,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.891938,0.000000,0.145244,0.00,0.00,inf,0.000000,0.0,0.

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
3,00:00,Eua Nba,Sacramento Kings,Golden State Warriors,235.5,1.88,1.0000,Over
5,21:00,Eua Nba,Orlando Magic,Washington Wizards,232.5,1.83,1.0000,Over
7,22:00,Eua Nba,Memphis Grizzlies,Utah Jazz,224.5,1.93,0.9607,Over
83,21:00,Eua Ncaa,Vermont,Dartmouth,132.5,1.91,0.9959,Over
115,23:15,Eua Ncaa,Vanderbilt,Boston College,146.5,1.91,0.8414,Over
176,23:00,Eua Nba,Denver Nuggets,Houston Rockets,215.5,1.83,0.9998,Over
